__No need to install modules outside of basic Python 3 modules__

In [1]:
import mysql.connector
import getpass
import pandas as pd

__Default username and password:
<br>
    root
    <br>
    CMSC461__

In [2]:
db_user = getpass.getpass('Enter username')
db_password = getpass.getpass('Enter password')

Enter username········
Enter password········


In [3]:
config = {
    'user': db_user,
    'password': db_password,
    'host': 'localhost',
    'port': 3306,
    'database': 'Project',
    'raise_on_warnings': True
}

In [4]:
# Connect to server
db_connection = mysql.connector.connect(**config)

In [5]:
# cursor of the output of queries
cur = db_connection.cursor()

__For all of the following cells, the default input/behavior for the queries is shown in the output of the cells__

In [44]:
# run this cell and insert your desired SQL query
# allows the user to access, populate, update, administer the SQL tables in the database
query = input("Insert your Query: ")

cur.execute(query)

result = cur.fetchall()
df = pd.DataFrame(result, columns= cur.column_names)
print(df)

Insert your Query: select first_name, last_name from driver
   first_name  last_name
0         Dan      James
1      Cheryl        Ann
2        Sara    Madison
3       Grace    Beverly
4        Mike      Henry
5        Ryan    Anthony
6        Liam     Oliver
7        Mary  Elizabeth
8         Don    Stevens
9       Frank  Alexander
10       Carl     Harold
11       Jude    Frances


In [6]:
# Run this cell to get the name of all the drivers
# Functional Requirement #1 in project
query = "Select first_name, last_name from driver"

cur.execute(query)

result = cur.fetchall()
df = pd.DataFrame(result, columns= cur.column_names)
print(df)

   first_name  last_name
0         Dan      James
1      Cheryl        Ann
2        Sara    Madison
3       Grace    Beverly
4        Mike      Henry
5        Ryan    Anthony
6        Liam     Oliver
7        Mary  Elizabeth
8         Don    Stevens
9       Frank  Alexander
10       Carl     Harold
11       Jude    Frances


In [30]:
# Run this cell to find the total miles traveled by all customers during a given date
# Functional Requirement #2 in project
date = input("Give a date (YYYY-MM-DD): ")
date = date + '%'
query = "Select sum(dist_change) from trip inner join trip_stops on trip.trajectory = trip_stops.trajectory where trip_stops.timestmp like %(arg)s"

cur.execute(query, {'arg': date})

result = cur.fetchall()
df = pd.DataFrame(result, columns= cur.column_names)
print(df)

Give a date (YYYY-MM-DD): 2015-02-01
  sum(dist_change)
0          25.5390


In [27]:
# Run this cell to find the most active driver (in terms of number of trips) during a given month 
# Functional Requirement #3 in project
month = input("Give a month (YYYY-MM): ")
month = month + '%'
query = "select count(trajectory), first_name, last_name from driver inner join trip on driver.driver_id = trip.driver inner join trip_stops using(trajectory) where is_last = 'yes' and timestmp like %(arg)s group by driver order by count(trajectory) desc limit 1" 

cur.execute(query, {'arg': month})

result = cur.fetchall()
df = pd.DataFrame(result, columns= cur.column_names)
print(df)

Give a month 'YYYY-MM': 2019-12
   count(trajectory) first_name last_name
0                  2        Dan     James


In [29]:
# Run this cell to find the most valuable customer in terms of money charged
# Functional Requirement #4 in project
query = "select * from (select sum(amount) as sums, card_no, first_name, last_name from customer inner join payment using (card_no) group by card_no) AS A group by sums order by sums desc limit 1"  

cur.execute(query)

result = cur.fetchall()
df = pd.DataFrame(result, columns= cur.column_names)
print(df)

     sums           card_no first_name last_name
0  600.00  1234567891111111       Most  Valuable


__This cell returns two values, the first being the total cost of all trips driven by a given driver,
<br>
the second being the amount of money credited to this driver__
<br>
__Additionally, it assumed that multiple customers may be making concurrent payments at the time the query is issued.__

In [31]:
# Run this cell to find the balance owed to a given driver
# the Driver must be entered in terms of their ID number so as to ensure that users cannot view information about drivers
# that they shouldn't know about
# Functional Requirement #5 in project
driver = input("Give a Driver's ID number: ")
query = "select sum(trip.cost) from driver inner join trip on driver.driver_id = trip.driver where driver_id = %(arg)s" 
cur.execute(query, {'arg': driver})
result = cur.fetchall()

df = pd.DataFrame(result, columns= cur.column_names)
print(df)
        
query2 = "select sum(payment.amount) from driver inner join payment on driver.card_no = payment.card_no where driver_id = %(arg)s"
cur.execute(query2, {'arg': driver})
result = cur.fetchall()

if not cur.rowcount:
    print("No payments have been made to this total yet.")
else:
    df = pd.DataFrame(result, columns= cur.column_names)
    print(df)

Give a Driver's ID number: ID8
  sum(trip.cost)
0          64.43
  sum(payment.amount)
0              -20.00


In [32]:
# Run this cell to find the vehicles that have traveled more miles in a given month than the previous month
# Functional Requirement #6 in project
month = input("Give a month (YYYY-MM):")
month = month + '-01'
query = "select * from((select vehicle, sum(dist_change) as current_month from vehicle inner join trip on vehicle.tag_info = trip.vehicle inner join trip_stops using (trajectory) where is_last = 'yes' and date(timestmp) >= date(%(arg)s) and date(timestmp) < date(%(arg)s) + interval 1 month group by vehicle) A inner join (select vehicle, sum(dist_change) as previous_month from vehicle inner join trip on vehicle.tag_info = trip.vehicle inner join trip_stops using (trajectory) where is_last = 'yes' and date(timestmp) >= date(%(arg)s) - interval 1 month and date(timestmp) < date(%(arg)s) group by vehicle) B on (A.current_month > B.previous_month and A.vehicle = B.vehicle))"
cur.execute(query, {'arg': month})
result = cur.fetchall()

df = pd.DataFrame(result, columns= cur.column_names)
print(df)

Give a month (YYYY-MM):2019-12
    vehicle current_month   vehicle previous_month
0  123456OH       12.9800  123456OH         1.0000


In [33]:
# Run this cell to Find the top 10 most popular trip trajectories (considering only their starting and ending stops) 
# in the last 24-hours. Gives a list of the most popular starting/ending points in the last 24 hours.
# Functional Requirement #7 in project 
query = "select is_last, street_no, street_name, city, state, zip, lat, lng, timestmp, count(*) from trip inner join trip_stops using (trajectory) where (is_last = 'yes' or dist_change = 0) group by trajectory, street_no, street_name, city, state, zip having (count(*) > 0 and timestmp >= NOW() - INTERVAL 1 DAY) order by count(*) desc limit 10"
cur.execute(query)
result = cur.fetchall()

if not cur.rowcount:
    print("No Trips have been made in the last 24 hours.")
else:
    df = pd.DataFrame(result, columns= cur.column_names)
    print(df)

  is_last street_no street_name         city state    zip       lat       lng  \
0      no      1234      Street  Catonsville    MD  21236  0.760123 -0.567903   
1      no      1234      Street  Catonsville    MD  21236  0.760123 -0.567903   
2      no      1234      Street  Catonsville    MD  21236  0.760123 -0.567903   
3      no      1234      Street  Catonsville    MD  21236  0.760123 -0.567903   
4     yes      1234       Query      Testing    MD  21236  0.760123 -0.567903   
5      no      1234      Street  Catonsville    MD  21236  0.760123 -0.567903   

              timestmp  count(*)  
0  2019-12-12 11:34:44         2  
1  2019-12-12 09:34:44         2  
2  2019-12-12 10:34:44         2  
3  2019-12-12 08:34:44         2  
4  2019-12-12 12:45:44         1  
5  2019-12-12 12:34:44         1  


In [33]:
# Run this cell to Find the closest available vehicle for a given location (specified by its coordinates).
# available = 1 means the vehicle is available
# Functional Requirement #8 in project 
lat = float(input("Enter a Latitude value: "))
long = float(input("Enter a Longitude value: "))

query = "select vehicle.tag_info, make, model, lat, lng, lugg_capacity, seating_capacity from vehicle inner join vehicle_last_loc using(tag_info) where vehicle.available = 1 order by abs(lat - %(arg)s) asc, abs(lng - %(arg2)s) asc limit 1;"
cur.execute(query, {'arg': lat, 'arg2': long})
result = cur.fetchall()

if not cur.rowcount:
    print("No results found.")
else:
    df = pd.DataFrame(result, columns= cur.column_names)
    print(df)

Enter a Latitude value: .77
Enter a Longitude value: .57
   tag_info    make    model   lat   lng  lugg_capacity  seating_capacity
0  123456MD  Toyota  Corolla  0.77 -0.57              5                 3


__More values may have to be added to the trips table for future testing of this cell as it was tested on 2019-12-12__

In [6]:
# Run this cell to Find the pairs of customers that took trips yesterday which ended at the same stop within 5
# minutes of each other. 
# returns a list of customers who arrived in similar locations at similar times
# Functional Requirement #9 in project 
query = "select first_name, last_name, name1, name2, lat1, lng1, A.timestmp, B.timestmp from((select first_name, last_name, timestmp, lat, lng, cust_id from customer inner join trip on customer.cust_id = trip.customer inner join trip_stops using (trajectory) where is_last = 'yes' and date(timestmp) = subdate(curdate(), 1) group by customer, lat, lng) A inner join (select first_name as name1, last_name as name2, timestmp, lat as lat1, lng as lng1, cust_id from customer inner join trip on customer.cust_id = trip.customer inner join trip_stops using (trajectory) where is_last = 'yes' and date(timestmp) = subdate(curdate(), 1) group by customer) B on (A.lng = B.lng1 and A.lat = B.lat1 and A.cust_id <> B.cust_id)) where abs(time_to_sec(timediff(time(A.timestmp), time(B.timestmp)))) < 300"
cur.execute(query)
result = cur.fetchall()

if not cur.rowcount:
    print("No results found.")
else:
    df = pd.DataFrame(result, columns= cur.column_names)
    print(df)

  first_name last_name name1  name2      lat1      lng1             timestmp  \
0       John       Doe   Ann  Smith  0.760123 -0.567903  2019-12-11 06:37:44   
1        Ann     Smith  John    Doe  0.760123 -0.567903  2019-12-11 06:38:44   

              timestmp  
0  2019-12-11 06:38:44  
1  2019-12-11 06:37:44  


In [7]:
# Run this cell to Find the customers that took trips with all the vehicles that a given customer took a trip with in a
# given week. 
# Functional Requirement #10 in project 
customer = input("Enter a customer's ID: ")
week_end = input("Give the date of the end of your desired week (YYYY-MM-DD):")
query = "select first_name, last_name, vehicle from customer inner join trip on customer.cust_id = trip.customer where vehicle in ( select vehicle from customer inner join trip on customer.cust_id = trip.customer inner join trip_stops using(trajectory) where dist_change = 0 and cust_id = %(arg)s and (date(timestmp) >= subdate(date(%(arg2)s),interval 1 week))) group by cust_id"

cur.execute(query, {'arg': customer, 'arg2': week_end})
result = cur.fetchall()

if not cur.rowcount:
    print("No Trips have been made in the last week.")
else:
    df = pd.DataFrame(result, columns= cur.column_names)
    print(df)

Enter a customer's ID: ID1
Give the date of the end of your desired week (YYYY-MM-DD):2019-12-11
  first_name  last_name   vehicle
0       John        Doe  123456MD
1      Query  10-Person  123456MD


In [10]:
cur.close()

False

In [11]:
db_connection.close()